In [1]:
import keras
import tensorflow as tf
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.models import Model
from keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt 
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from flask import Flask, render_template, request, send_file
from PIL import Image
import os
import pickle
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from googletrans import Translator
from gtts import gTTS

app = Flask(__name__)

with open("/Users/bharathmogalapu/Downloads/tokenizer1.pkl", 'rb') as f:
   tokenizer = pickle.load(f)
max_length=35
from keras.models import load_model 
model=load_model('/Users/bharathmogalapu/Downloads/archive/model90.h5')


def idx_to_word(integer, tokenizer):
    for word,index, in tokenizer.word_index.items():
        if index == integer:
            return word
    return None


def predict_caption(model, image, tokenizer, max_length):
    # add start tag for generation process
    in_text = 'startseq'
    # iterate over the max length of sequence
    for i in range(max_length):
        # encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # pad the sequence
        sequence = pad_sequences([sequence], max_length)
        # predict next word
        yhat = model.predict([image, sequence], verbose=0)
        # get index with high probability
        yhat = np.argmax(yhat)
        # convert index to word
        word = idx_to_word(yhat, tokenizer)
        # stop if word not found
        if word is None:
            break
        # append word as input for generating next word
        in_text += " " + word
        # stop if we reach end tag
        if word == 'endseq':
            break
    return in_text[8:-6]


def new_caption(image_path, model, tokenizer, max_length):
    """
    Generates a caption for an image using a trained image captioning model.

    Args:
        image_path (str): Path to the image file.
        model: The trained image captioning model.
        tokenizer: The tokenizer used to encode text for the model.
        max_length: The maximum length of the generated caption.

    Returns:
        str: The generated caption.
    """

    # Load the Model
    x_model = Xception()

# Restructure model
    x_model = Model(inputs = x_model.inputs , outputs = x_model.layers[-2].output)

    # Display the image
    img = load_img(image_path)
    plt.imshow(img)

    # Load and preprocess the image
    image = load_img(image_path, target_size=(299, 299))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    # Extract features from the image using VGG16
    feature = x_model.predict(image, verbose=0)

    # Generate caption using the trained model
    caption = predict_caption(model, feature, tokenizer, max_length)

    return caption

# Function to translate text
def translate_text(text, dest_language):
    translator = Translator()
    translated_text = translator.translate(text, dest=dest_language)
    return translated_text.text

# Function to convert text to speech
def text_to_speech(text, language):
    tts = gTTS(text=text, lang=language, slow=False)
    tts.save("static/caption_audio.mp3")

@app.route('/', methods=['GET'])
def index():
    return render_template('main.html')

@app.route('/', methods=['POST'])
def predict():
    # Get the uploaded image file
    imagefile = request.files['imagefile']
    # Save the image file
    image_path = './static/' + imagefile.filename
    imagefile.save(image_path)
    # Get the selected language from the form
    selected_language = request.form['language']
    # Generate caption for the uploaded image
    caption = new_caption(image_path,model,tokenizer,max_length)
    # Translate the generated caption
    translated_caption = translate_text(caption, selected_language)
    # Convert caption to speech
    text_to_speech(translated_caption, selected_language)
    return render_template('display3.html', prediction=translated_caption, path=image_path)

# Route to serve the audio file
@app.route('/caption_audio')
def caption_audio():
    return send_file("static/caption_audio.mp3", as_attachment=True)

if __name__ == '__main__':
    app.run()

/Users/bharathmogalapu/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [19/Jun/2024 17:25:27] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Jun/2024 17:25:27] "GET /main.css HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Jun/2024 17:25:27] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Jun/2024 17:26:02] "GET /static/image.png HTTP/1.1" 200 -
